In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# data

In [3]:
mnist = input_data.read_data_sets("../CNN/MNIST_data",one_hot=True)

Extracting ../CNN/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../CNN/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../CNN/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../CNN/MNIST_data\t10k-labels-idx1-ubyte.gz


# 参数

In [4]:
lr = 0.001
training_iters = 100000
batch_size = 128

In [32]:
n_inputs = 28
n_steps = 28
n_hidden_units = 128
n_classes = 10

# placeholder

In [52]:
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_classes])

# weights and biases

In [53]:
weights = {
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
biases = { 
    'in' :tf.Variable(tf.constant(0.1,shape = [n_hidden_units,])),
    'out':tf.Variable(tf.constant(0.1,shape = [n_classes,]))
}

In [57]:
def RNN(X,weights,biases):
    # hidden layer for input to cell
    X = tf.reshape(X,[-1,n_inputs])
    X_in = tf.matmul(X,weights['in']) + biases['in']
    X_in = tf.reshape(X_in,[-1,n_steps,n_inputs])
    
    #cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
    #忘记门的biases是1，初始时不忘记主线内容，因为时lstm，会产生两个states：c_state,m_state,共同组成一个元组
    init_state = lstm_cell.zero_state(batch_size,dtype = tf.float32)
    
    outputs,states = tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=init_state,time_major=False)
    #最后的计算结果：outputs包含了每一步的输出，states时最后一步的state，time_major指的时对于输入X_in，time_step是不是在它的第一个维度上，如果不是则填false
    
    #outputs
#     results = tf.matmul(states[1],weights['out'])+biases['out']
    #states[1]是m_state,在本例中也等于outputs[-1](最后一个输出)
    
#     第二种求结果的方法
    outputs = tf.unpack(tf.transpose(outputs,[1,0,2]))
#     把outputs的维度的前两个维度进行了交换：[batch_size,n_steps,]
    results = tf.matmul(outputs[-1],weights['out'])+biases['out']
    
    return results

In [58]:
pred = RNN(X,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1269, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2506, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [25]:
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [38]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = np.reshape(batch_xs, [-1, n_steps, n_inputs])
        sess.run(train_op,feed_dict = {
            X:batch_xs,
            y:batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy,feed_dict={
                X:batch_xs,
                y:batch_ys,
            }))
        step += 1

Instructions for updating:
Use `tf.global_variables_initializer` instead.


InvalidArgumentError: Shape [-1,28,28] has negative dimensions
	 [[Node: Placeholder_4 = Placeholder[dtype=DT_FLOAT, shape=[?,28,28], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_4', defined at:
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\stuian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\stuian\AppData\Local\Continuum\anac